# **Exercicios**

1. encontrar inconsistencias entre os campos tx_tip_pgto_tran, nm_lncd, tx_idfr_pcl, qt_pcl_tran
2. tratar o campo tx_idfr_pcl (deve ser o numero da parcela que está sendo paga), removendo strings e trazendo apenas integers;
3. tratar o campo vl_moen (quando tx_tip_lcto for DEBITO, o vl_moen deve ser negativo, e quando for CREDITO, deve ser positivo) [observar valores já negativos - tirar o módulo da coluna antes de transformar]];
4. tratar o campo nm_tran (ifood e 99app - padronizar a escrita)

## **Initial Configs**:

In [1]:
!python.exe -m pip install --upgrade pip
!pip3 install pyspark

  Using cached pip-23.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
  Using cached pyspark-3.3.1.tar.gz (281.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845533 sha256=cf5911cf99fc008fc04a46b599c897eff24ec31ea0625e05327c383959fcf69a
  Stored in directory: c:\users\017924631\appdata\local\pip\cache\wheels\0f\f0\3d\517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark


In [24]:
from pyspark.sql import SparkSession # importing pyspark library

from IPython.core.display import HTML
display(HTML('<style>pre { white-space: pre !important; }</style>'))

In [7]:
spark = SparkSession.builder.getOrCreate() # instanciando uma SparkSession

## **Importando dados**:

In [5]:
def find_df(name):
    import os

    for root, dirs, files in os.walk(os.getcwd().replace('\\','/')):
        if name in files:
            return os.path.join(root, name)
        else:
            return None


In [4]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



### Dicionário da tabela:

In [4]:
import pandas
import numpy as np

from pyspark.sql.types import *

In [9]:
df_path = find_df('dict_tran_crt.csv')
df_dict = pandas.read_csv(df_path, sep=';')
df_dict.head(3)

,col_name,data_type,comment
0,ts_atl_tran,timestamp,Data e hora de atualizacao das informacoes de ...
1,cd_ct_crt,char(100),Identifica de forma unica a conta pagamento po...
2,cd_idfr_tran,char(100),Codigo ou identificador unico prestado pela in...


In [46]:

df_path = find_df('tran_anonimizada.csv')

df_pandas = pandas.read_csv(df_path)

df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ts_atl_tran       200 non-null    object 
 1   cd_idfr_tran      200 non-null    object 
 2   tx_idfr_crt       199 non-null    float64
 3   nm_lncd           194 non-null    object 
 4   nm_tran           199 non-null    object 
 5   cd_idfr_fat       195 non-null    object 
 6   tx_tip_lcto       199 non-null    object 
 7   tx_tip_tran       199 non-null    object 
 8   tx_adc_tran       199 non-null    object 
 9   tx_tip_pgto_tran  199 non-null    object 
 10  tx_tip_tarf       199 non-null    object 
 11  tx_adc_tip_tarf   199 non-null    object 
 12  tx_tip_opr_crd    199 non-null    object 
 13  tx_adc_opr_crd    199 non-null    object 
 14  tx_idfr_pcl       199 non-null    object 
 15  qt_pcl_tran       199 non-null    float64
 16  vl_moen           199 non-null    float64
 1

In [51]:
def column_nanfix(dataframe: pandas.DataFrame) -> pandas.DataFrame:
    '''
    Quando uma coluna no pandas dataframe contém
    valores NaN. O PySpark tem problemas para conversão.
    Portanto, solucionaremos este bug convertendo os valores
    NaN em '', para fazermos a conversão.
    '''
    df_header = list(dataframe.columns)
    columns_nan =   [ 
                        column for column in df_header 
                            if dataframe[column].isnull()
                            .values.any() 
                    ]


    for column in columns_nan:
        dataframe[column] = dataframe[column].fillna('')
    
    return dataframe

def column_change_datatype(dataframe: pandas.DataFrame, df_list: list) -> pandas.DataFrame:
    df_header = list(dataframe.columns)
    for column in df_header:
        if column in df_list:
            dataframe[column] = dataframe[column].astype(str)
    return dataframe


In [54]:
df_dict_columns         = [ columns for columns in df_dict['col_name'] ]
df_dict_dtypes          = [ dtypes for dtypes in df_dict['data_type'] ]
df_dict_columns_dtypes  = dict(zip( df_dict_columns, df_dict_dtypes ))
df_dict_columns_str     = [ chave for chave, valor in 
                          df_dict_columns_dtypes.items() if 'char' in valor ]

df_pandas_strfix        = column_change_datatype(df_pandas, df_dict_columns_str)
df_pandas_nanfix        = column_nanfix(df_pandas_strfix)

In [56]:
df_raw = spark.createDataFrame(df_pandas)
df_raw.show()

c:\Users\017924631\Desktop\working-tree\ibm-school-of-data_engineering\6-spark\ambvir\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\017924631\Desktop\working-tree\ibm-school-of-data_engineering\6-spark\ambvir\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


TypeError: field qt_pcl_tran: Can not merge type <class 'pyspark.sql.types.DoubleType'> and <class 'pyspark.sql.types.StringType'>

In [26]:
from pyspark.sql import functions as f

df_raw =  spark.read.csv (
            find_df('tran_anonimizada.csv'), 
            header=True, 
            sep=',', 
            inferSchema=True, 
            quote='"', 
            escape='"'
)

for col in df_raw.columns:
    df_raw = df_raw.withColumn(col, f.trim(f.col(col)))

df_raw.show(vertical=False)

+-------------------+--------------------+-----------+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+----------------+-----------+--------------------+-------------------+--------------------+------------------+-----------+-------+-------+--------------+----------+-----------+-----------+----------+--------------+-------------------+--------------------+----------+
|        ts_atl_tran|        cd_idfr_tran|tx_idfr_crt|          nm_lncd|             nm_tran|         cd_idfr_fat|tx_tip_lcto|         tx_tip_tran|         tx_adc_tran|tx_tip_pgto_tran|tx_tip_tarf|     tx_adc_tip_tarf|     tx_tip_opr_crd|      tx_adc_opr_crd|       tx_idfr_pcl|qt_pcl_tran|vl_moen|vl_extr|cd_moe_vl_extr|   dt_tran|dt_lcto_fat|cd_ctgr_cia|cd_tip_pss|cd_tip_mvt_opb|         nm_ifc_dst|     nm_prmo_arq_ogm|dt_arq_ogm|
+-------------------+--------------------+-----------+-----------------+--------------------+--------------------+-----------+----

In [14]:
df_raw

DataFrame[ts_atl_tran: timestamp, cd_idfr_tran: string, tx_idfr_crt: string, nm_lncd: string, nm_tran: string, cd_idfr_fat: string, tx_tip_lcto: string, tx_tip_tran: string, tx_adc_tran: string, tx_tip_pgto_tran: string, tx_tip_tarf: string, tx_adc_tip_tarf: string, tx_tip_opr_crd: string, tx_adc_opr_crd: string, tx_idfr_pcl: string, qt_pcl_tran: string, vl_moen: string, vl_extr: string, cd_moe_vl_extr: string, dt_tran: string, dt_lcto_fat: string, cd_ctgr_cia: string, cd_tip_pss: int, cd_tip_mvt_opb: string, nm_ifc_dst: string, nm_prmo_arq_ogm: string, dt_arq_ogm: string]